In [1]:
import geopandas as gpd
import os
import pandas as pd
import folium
from folium.plugins import MarkerCluster, Search
import numpy as np

In [2]:
def merge_csv_into_gdf():
    gdf = None
    for (dir,_,file) in os.walk("./data/"):
        print(file)
        for f in file:
            if gdf is None:
                gdf = pd.read_csv(dir+f)
            else:
                gdf= pd.concat([gdf,pd.read_csv(dir+f)],ignore_index=True)

    gdf = gpd.GeoDataFrame(gdf,geometry=gpd.points_from_xy(gdf.lng,gdf.lat),crs="epsg:4326")
    return gdf

In [3]:
all_gdf = merge_csv_into_gdf()
all_gdf

['traditional_w_coordinate_27500-27600.csv', 'traditional_w_coordinate_15100-15200.csv', 'traditional_w_coordinate_13900-14000.csv', 'traditional_w_coordinate_1700-1800.csv', 'traditional_w_coordinate_14500-14600.csv', 'traditional_w_coordinate_18300-18400.csv', 'traditional_w_coordinate_1100-1200.csv', 'traditional_w_coordinate_26100-26200.csv', 'traditional_w_coordinate_7600-7700.csv', 'traditional_w_coordinate_5000-5100.csv', 'traditional_w_coordinate_3900-4000.csv', 'traditional_w_coordinate_8000-8100.csv', 'traditional_w_coordinate_1800-1900.csv', 'traditional_w_coordinate_10100-10200.csv', 'traditional_w_coordinate_19400-19500.csv', 'traditional_w_coordinate_22500-22600.csv', 'traditional_w_coordinate_23100-23200.csv', 'traditional_w_coordinate_7700-7800.csv', 'traditional_w_coordinate_18000-18100.csv', 'traditional_w_coordinate_11500-11600.csv', 'traditional_w_coordinate_1600-1700.csv', 'traditional_w_coordinate_7100-7200.csv', 'traditional_w_coordinate_3000-3100.csv', 'traditio

,Unnamed: 0,번호,시장명,상호명,주소,연락처,취급품목(업종),lat,lng,geometry
0,28876,319,화양제일골목시장(화양제일시장),화양식품,서울특별시 광진구 아차산로29길 59(화양동)1층,02-465-6374,두부,37.543482,127.069282,POINT (127.06928 37.54348)
1,28877,318,화양제일골목시장(화양제일시장),레드크라운,서울특별시 광진구 아차산로29길 56(화양동)1층,02-3409-5545,도소매(케이스),37.543187,127.069398,POINT (127.06940 37.54319)
2,28878,317,화양제일골목시장(화양제일시장),해안수산,서울 광진구 동일로24길 70(화양동),NaN,농축수산품,37.542833,127.068790,POINT (127.06879 37.54283)
3,28879,316,화양제일골목시장(화양제일시장),신토불이청과,서울특별시 광진구 동일로24길 64(화양동)1층,02-463-6740,농산물,37.542940,127.068531,POINT (127.06853 37.54294)
4,28880,315,화양제일골목시장(화양제일시장),제일상회,서울특별시 광진구 아차산로29길 59(화양동) 1층,02-465-3067,농산물,37.543482,127.069282,POINT (127.06928 37.54348)
...,...,...,...,...,...,...,...,...,...,...
27803,17530,11124,성내전통시장,행운축산,서울특별시 강동구 풍성로37가길 17(성내동) 1층,02-489-5221,소매(식육),37.533297,127.127313,POINT (127.12731 37.53330)
27804,17531,11123,성내전통시장,속옷할인매장,서울특별시 강동구 천호대로162길 50(성내동) 속옷할인매장,NaN,소매(속옷),37.534700,127.127870,POINT (127.12787 37.53470)
27805,17532,11122,성내전통시장,김치전문,서울특별시 강동구 천호대로162길 73(성내동)1층 3호,NaN,소매(김치),37.533393,127.127705,POINT (127.12770 37.53339)
27806,17533,11121,성내전통시장,안성꽃상회,서울특별시 강동구 천호대로162길 65(성내동) (성내골목시장)안성꽃상회 1층 102호,02-484-2581,생화 양곡,37.533990,127.127835,POINT (127.12784 37.53399)


# Make a huge map

In [4]:
m = folium.Map([37.533297,127.127313],zoom_start=12)
folium.GeoJson(all_gdf).add_to(m)
m.save("./all_market.html")

# try marker cluster

In [5]:
gdf_for_marker = all_gdf[all_gdf["lat"].notna()]
locs = gdf_for_marker[["lat","lng"]]
lats = np.array(locs["lat"])
lngs = np.array(locs["lng"])
locations = list(zip(lats,lngs))
popups = gdf_for_marker[["시장명","상호명","취급품목(업종)","주소"]]

m = folium.Map([37.533297,127.127313],
                tiles="Cartodb Positron",
               zoom_start=12)

marker_cluster = MarkerCluster(
    name="Traditional Markets",
    overlay=True,
    control=True
)

popups_lst = popups.values.tolist()
for ((mk_name,store_name,category,adrs),(lat,lng)) in zip(popups_lst,locations):
    cur_loc = lat,lng
    marker = folium.Marker(location=cur_loc,name=store_name+"/"+str(category),cat=category)
    popup = f'<h3>{store_name},{category}</h3><b>시장</b>:{mk_name}<br><b>주소</b>:{adrs}'
    folium.Popup(popup,max_width=300).add_to(marker)
    marker_cluster.add_child(marker)

marker_cluster.add_to(m)

name_search = Search(
    layer=marker_cluster,
    search_label="name",
    placeholder="이름을 입력하세요",
    collapsed=False,
).add_to(m)



folium.LayerControl().add_to(m)
m.save("./market_clustered.html")

# Split by market

In [6]:
gdf_dict = dict(tuple(gdf_for_marker.groupby("시장명")))
for key,val in gdf_dict.items():
    m = folium.Map